- welcome to the final project of the Udacity Self-Driving Car Engineer Nanodegree program
- the goal of this project is to enable Carla to drive around our test track using waypoint navigation
- Carla will do this while avoiding obstacles and stopping at traffic lights


- waypoints are simply an ordered set of coordinates that Carla uses to plan a path around the track
- each of these waypoints also has an associated target velocity
- Carla's planning subsystem updates the target velocity for the waypoints ahead of the vehicle depending on the desired vehicle behavior
- Carla's control subsystem actuates the throttle, steering, and break to successfully navigate the waypoints with the correct target velocity


- over the course of the project, you'll be implementing components of the perception, planning, and control subsystems
- in the perception subsystem, you'll implement traffic light detection and obstacle detection
- in the planning subsystem, you'll implement a node called the waypoint updater
  - this node sets the target velocity for each waypoint based on the upcoming traffic lights and obstacles
    - for example, if you see a red traffic light on the horizon, you'll want to set decelerating velocities at the nodes leading up to that traffic light
- in the control subsystem, you'll implement a drive by wire ROS node that takes target trajectory information is input, in sense, control commands to navigate the vehicle


- we've provided a ROS framework for these nodes, as well as a version of the simulator that includes traffic lights and obstacles
- the ROS framework we've provided works with both the Udacity simulator and with Karla, so you can develop your project on the simulator before submitting the same code to be tested on the vehicle

# Project Setup

- the project will require the use of Ubuntu Linux (the operating system of Carla) and a new simulator
- if you prefer not to use the provided in-browser Workspace, follow the steps below to get set up:
  - because ROS is used, you will need to use Ubuntu to develop and test your project code
  - you may use:
    - Ubuntu 14.04 with ROS Indigo
    - Ubuntu 16.04 with ROS Kinetic


- you are welcome to use your own Ubuntu installation or virtual machine (unsupported), or you can use the VM provided in Your Virtual Machine in the "Introduction to ROS" lesson
- the provided VM has ROS and Dataspeed DBW installed already
- Windows 10 users - your fellow students have suggested that the best local option is to use the VM for ROS, while running the simulator natively (and making sure to open ports between the two for communication)
- the project repo can be found [here](https://github.com/udacity/CarND-Capstone)
  - clone or download the project code before the next sections so you can follow along with the code descriptions!
  - in the README, you should be able find any additional dependencies needed for the project
- the system integration project uses its own simulator which will interface with your ROS code and has traffic light detection
  - you can download the simulator [here](https://github.com/udacity/CarND-Capstone/releases)
  - to improve performance while using a VM, we recommend downloading the simulator for your host operating system and using this outside of the VM
  - you will be able to run project code within the VM while running the simulator natively in the host using port forwarding on port `4567`
  - for more information on how to set up port forwarding, see the end of the classroom concept [here](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/files/Port+Forwarding.pdf)


- the rubric for this project is fairly simple - does the vehicle successfully navigate the track? 
- note that the latest version of the simulator has two test tracks:
  - a highway test track with traffic lights
  - a testing lot test track similar to where we will run Carla
- to use the second test lot, you will need to update your code to specify a new set of waypoints
  - we'll discuss how to do this in a later lesson
- additionally, the first track has a toggle button for camera data
  - many students have experienced latency when running the simulator together with a virtual machine, and leaving the camera data off as you develop the car's controllers will help with this issue
- finally, the simulator displays vehicle velocity in units of mph - however, all values used within the project code are use the metric system (m or m/s), including current velocity data coming from the simulator

# System Architecture Diagram

- for this project, you'll be writing ROS nodes to implement core functionality of the autonomous vehicle system, including traffic light detection, control, and waypoint following!
- you will test your code using a simulator, and when you are ready, your group can submit the project to be run on Carla


- the following is a system architecture diagram showing the ROS nodes and topics used in the project
- you can refer to the diagram throughout the project as needed
- the ROS nodes and topics shown in the diagram are described briefly in the Code Structure section below, and more detail is provided for each node in later classroom concepts of this lesson

<img src="resources/system_architecture_diagram.png"/>

# Code Structure

- below is a brief overview of the repo structure, along with descriptions of the ROS nodes
- the code that you will need to modify for the project will be contained entirely within the `(path_to_project_repo)/ros/src/` directory
- within this directory, you will find the following ROS packages:


#### (path_to_project_repo)/ros/src/tl_detector/

- this package contains the traffic light detection node: `tl_detector.py`
- this node takes in data from the `/image_color`, `/current_pose`, and `/base_waypoints` topics and publishes the locations to stop for red traffic lights to the `/traffic_waypoint` topic


- the `/current_pose` topic provides the vehicle's current position, and `/base_waypoints` provides a complete list of waypoints the car will be following


- you will build both a traffic light detection node and a traffic light classification node
- traffic light detection should take place within `tl_detector.py`, whereas traffic light classification should take place within `../tl_detector/light_classification_model/tl_classfier.py`

<img src="resources/tl-detector-ros-graph.png"/>


#### path_to_project_repo)/ros/src/waypoint_updater/

- this package contains the waypoint updater node: `waypoint_updater.py`
- the purpose of this node is to update the target velocity property of each waypoint based on traffic light and obstacle detection data
- this node will subscribe to the `/base_waypoints`, `/current_pose`, `/obstacle_waypoint`, and `/traffic_waypoint` topics, and publish a list of waypoints ahead of the car with target velocities to the `/final_waypoints` topic

<img src="resources/waypoint-updater-ros-graph.png"/>


#### (path_to_project_repo)/ros/src/twist_controller/

- Carla is equipped with a drive-by-wire (dbw) system, meaning the throttle, brake, and steering have electronic control
- this package contains the files that are responsible for control of the vehicle: the `node dbw_node.py` and the `file twist_controller.py`, along with a pid and lowpass filter that you can use in your implementation


- the `dbw_node` subscribes to the `/current_velocity` topic along with the `/twist_cmd` topic to receive target linear and angular velocities
- additionally, this node will subscribe to `/vehicle/dbw_enabled`, which indicates if the car is under dbw or driver control
- this node will publish throttle, brake, and steering commands to the `/vehicle/throttle_cmd`, `/vehicle/brake_cmd`, and `/vehicle/steering_cmd` topics

<img src="resources/dbw-node-ros-graph.png"/>


- in addition to these packages you will find the following, which are not necessary to change for the project
- the `styx` and `styx_msgs` packages are used to provide a link between the simulator and ROS, and to provide custom ROS message types:
  - `(path_to_project_repo)/ros/src/styx/`
    - a package that contains a server for communicating with the simulator, and a bridge to translate and publish simulator messages to ROS topics
  - `(path_to_project_repo)/ros/src/styx_msgs/`
    - a package which includes definitions of the custom ROS message types used in the project
  - `(path_to_project_repo)/ros/src/waypoint_loader/`
    -  package which loads the static waypoint data and publishes to `/base_waypoints`
  - `(path_to_project_repo)/ros/src/waypoint_follower/`
    - a package containing code from [Autoware](https://github.com/CPFL/Autoware) which subscribes to `/final_waypoints` and publishes target vehicle linear and angular velocities in the form of twist commands to the `/twist_cmd` topic

# Suggested Order of Project Development

- because you will be writing code across several packages with some nodes depending on messages published by other nodes, we suggest completing the project in the following order:
  - Waypoint Updater Node (Partial): complete a partial waypoint updater which subscribes to `/base_waypoints` and `/current_pose` and publishes to `/final_waypoints`
  - DBW Node: once your waypoint updater is publishing `/final_waypoints`, the `waypoint_follower node` will start publishing messages to the `/twist_cmd` topic
    - at this point, you have everything needed to build the `dbw_node`
    - after completing this step, the car should drive in the simulator, ignoring the traffic lights
  - Traffic Light Detection: this can be split into 2 parts:
    - Detection: detect the traffic light and its color from the `/image_color`
      - the topic `/vehicle/traffic_lights` contains the exact location and status of all traffic lights in simulator, so you can test your output
    - Waypoint publishing: once you have correctly identified the traffic light and determined its position, you can convert it to a waypoint index and publish it
  - Waypoint Updater (Full): use `/traffic_waypoint` to change the waypoint target velocities before publishing to `/final_waypoints`
    - your car should now stop at red traffic lights and move when they are green


- in the next few classroom concepts, we'll cover each component of the project
- you can use these concepts as a guide for the suggested order of project completion, but feel free to ignore these steps and implement the project how you please!

# Waypoint Updater Node (Partial)

### Waypoint Updater Node Overview

- before the car can move in the simulator, it will be necessary to write a first version of the `waypoint_updater` node
- you can find class and method stubs for the node in `(path_to_project_repo)/ros/src/waypoint_updater/waypoint_updater.py`

- the eventual purpose of this node is to publish a fixed number of waypoints ahead of the vehicle with the correct target velocities, depending on traffic lights and obstacles
- the goal for the first version of the node should be simply to subscribe to the topics `/base_waypoints` and `/current_pose` and publish a list of waypoints to `/final_waypoints`


- the `/base_waypoints` topic publishes a list of all waypoints for the track, so this list includes waypoints both before and after the vehicle (note that the publisher for `/base_waypoints` publishes only once)
- for this step in the project, the list published to `/final_waypoints` should include just a fixed number of waypoints currently ahead of the vehicle:
  - the first waypoint in the list published to `/final_waypoints` should be the first waypoint that is currently ahead of the car
  - the total number of waypoints ahead of the vehicle that should be included in the `/final_waypoints` list is provided by the `LOOKAHEAD_WPS` variable in `waypoint_updater.py`

### Waypoint Message Descriptions

- this section will demonstrate use of `rostopic` and `rosmsg` to learn more about the messages being transmitted on `/base_waypoints` and `/final_waypoints`


- from the code in `waypoint_updater.py`, we can see that both the `/final_waypoints` and `/base_waypoints` topics have message type `Lane`
- you can look at the details about this message type in `<path_to_project_repo>/ros/src/styx_msgs/msg/`, but this can also be done from the command line after launching the ROS project using `ros topic` and `rosmsg` as follows:
  - after opening a new terminal window and sourcing `devel/setup.bash`, you can see see all topics by executing:

```shell
rostopic list
```

- you should see `/final_waypoints` among the topics
- to get info about the message type being used in `/final_waypoints`, execute:

```shell
rostopic info /final_waypoints
```

- next, execute:

```shell
rosmsg info styx_msgs/Lane
```


- it provides the following message information:

```
std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
styx_msgs/Waypoint[] waypoints
  geometry_msgs/PoseStamped pose
    std_msgs/Header header
      uint32 seq
      time stamp
      string frame_id
    geometry_msgs/Pose pose
      geometry_msgs/Point position
        float64 x
        float64 y
        float64 z
      geometry_msgs/Quaternion orientation
        float64 x
        float64 y
        float64 z
        float64 w
  geometry_msgs/TwistStamped twist
    std_msgs/Header header
      uint32 seq
      time stamp
      string frame_id
    geometry_msgs/Twist twist
      geometry_msgs/Vector3 linear
        float64 x
        float64 y
        float64 z
      geometry_msgs/Vector3 angular
        float64 x
        float64 y
        float64 z
```

- from here you can see that the messages contain a `header` and a `Waypoint` list named `waypoints`
- each waypoint has `pose` and `twist` data
- going further, you can see that `twist.twist` data contains 3D `linear` and `angular` velocities
- for more information about twist messages, [see documentation here](http://docs.ros.org/jade/api/geometry_msgs/html/msg/Twist.html)
- check to be sure you can explore the messages sent in `/base_waypoints` and `/current_pose` as well using the steps above!

### Lane message example

- as a use-case example, given a single `styx_msgs/Lane` message `my_lane_msg`, you can access the x direction linear velocity of the first waypoint in Python with:

```python
my_lane_msg[0].twist.twist.linear.x
```

- note that the coordinates for linear velocity are vehicle-centered, so only the x-direction linear velocity should be nonzero

### Topics and message types

- for convenience, we have provided the following table with topic and message info for this step of the project:
  - topic: `/base_waypoints`
    - msg_type: `styx_msgs/Lane`
    - notes: waypoints as provided by a static .csv file
  - topic: `/current_pose`
    - msg_type: `geometry_msgs/PoseStamped`
    - notes: current position of the vehicle, provided by the simulator or localization
  - topic: `/final_waypoints`
    - msg_type: `styx_msgs/Lane`
    - notes: this is a subset of `/base_waypoints`
      - the first waypoint is the one in `/base_waypoints` which is closest to the car

# DBW Node

### DBW Node Overview

- once messages are being published to `/final_waypoints`, the vehicle's waypoint follower will publish twist commands to the `/twist_cmd` topic
- the goal for this part of the project is to implement the drive-by-wire node (`dbw_node.py`) which will subscribe to `/twist_cmd` and use various controllers to provide appropriate throttle, brake, and steering commands
- these commands can then be published to the following topics: `/vehicle/throttle_cmd`, `/vehicle/brake_cmd`, `/vehicle/steering_cmd`


- since a safety driver may take control of the car during testing, you should not assume that the car is always following your commands
- if a safety driver does take over, your PID controller will mistakenly accumulate error, so you will need to be mindful of DBW status
- the DBW status can be found by subscribing to `/vehicle/dbw_enabled`


- when operating the simulator please check DBW status and ensure that it is in the desired state
- DBW can be toggled by clicking "Manual" in the simulator GUI


- all code necessary to implement the drive-by-wire node can be found in the package: `(path_to_project_repo)/ros/src/twist_controller`

### Twist controller package files

- within the twist controller package, you will find the following:
  - `dbw_node.py`
    - this python file implements the `dbw_node` publishers and subscribers
    - you will need to write ROS subscribers for the `/current_velocity`, `/twist_cmd`, and `/vehicle/dbw_enabled topics`
    - this file also imports the `Controller` class from `twist_controller.py` which will be used for implementing the necessary controllers
    - the function used to publish throttle, brake, and steering is `publish`
      - note that throttle values passed to `publish` should be in the range 0 to 1, although a throttle of 1 means the vehicle throttle will be fully engaged
      - brake values passed to `publish` should be in units of torque (N\*m) - the correct values for brake can be computed using the desired acceleration, weight of the vehicle, and wheel radius
  - `twist_controller.py`
    - this file contains a stub of the `Controller` class
    - you can use this class to implement vehicle control
    - for example, the control method can take twist data as input and return throttle, brake, and steering values
    - within this class, you can import and use the provided `pid.py` and `lowpass.py` if needed for acceleration, and `yaw_controller.py` for steering
    - note that it is not required for you to use these, and you are free to write and import other controllers
  - `yaw_controller.py`
    - a controller that can be used to convert target linear and angular velocity to steering commands
  - `pid.py`
    - a generic PID controller that can be used in `twist_controller.py`
  - `lowpass.py`
    - a generic low pass filter that can be used in `twist_controller.py`
  - `dbw_test.py`
    - you can use this file to test your DBW code against a bag recorded with a reference implementation
    - the bag can be found at https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/files/reference.bag.zip
    - detailed use instructions can be found in the `dbw_test.py` file

### Testing with the simulator

- Highway
  - at this point in the project, it is unnecessary to use camera data
  - if you are experiencing system latency or dropped ROS messages, be sure to keep the camera toggle unchecked in the first test track in the simulator
- Test Lot
  - the `CarND-Capstone/ros/src/waypoint_loader/launch/waypoint_loader.launch` file is set up to load the waypoints for the first track
  - to test using the second track, you will need to change 

```xml
<param name="path" value="$(find styx)../../../data/wp_yaw_const.csv"/>
``` 

  - to use the `churchlot_with_cars.csv` as follows:
    
```xml
<param name="path" value="$(find styx)../../../data/churchlot_with_cars.csv"/>
```

- note that the second track does not send any camera data

### Important

- `dbw_node.py` is currently set up to publish steering, throttle, and brake commands at 50Hz
- the DBW system on Carla expects messages at this frequency, and will disengage (reverting control back to the driver) if control messages are published at less than 10Hz
- this is a safety feature on the car intended to return control to the driver if the software system crashes
- you are welcome to modify how the `dbw_node.py` code is structured, but please ensure that control commands are published at 50Hz


- additionally, although the simulator displays speed in mph, all units in the project code use the metric system, including the units of messages in the `/current_velocity` topic (which have linear velocity in m/s)


- finally, Carla has an automatic transmission, which means the car will roll forward if no brake and no throttle is applied
- to prevent Carla from moving requires about 700 Nm of torque
- in the walkthrough, only 400 Nm of torque is applied to hold the vehicle stationary
  - this turns out to be slightly less than the amount of force needed, and Carla will roll forward with only 400Nm of torque
  - to prevent Carla from moving you should apply approximately 700 Nm of torque

# Traffic Light Detection Node

### Traffic Light Detection Node Overview

- once the vehicle is able to process waypoints, generate steering and throttle commands, and traverse the course, it will also need stop for obstacles
- traffic lights are the first obstacle that we'll focus on


- the traffic light detection node (`tl_detector.py`) subscribes to four topics:
  - `/base_waypoints` provides the complete list of waypoints for the course
  - `/current_pose` can be used to determine the vehicle's location
  - `/image_color` which provides an image stream from the car's camera
    - these images are used to determine the color of upcoming traffic lights
  - `/vehicle/traffic_lights` provides the (x, y, z) coordinates of all traffic lights
- the node should publish the index of the waypoint for nearest upcoming red light's stop line to a single topic: `/traffic_waypoint`


- for example, if `waypoints` is the complete list of waypoints, and an upcoming red light's stop line is nearest to `waypoints[12]`, then `12` should be published `/traffic_waypoint`
- this index can later be used by the waypoint updater node to set the target velocity for `waypoints[12]` to `0` and smoothly decrease the vehicle velocity in the waypoints leading up to `waypoints[12]`


- the permanent `(x,y)` coordinates for each traffic light's stop line are provided by the `config` dictionary, which is imported from the `traffic_light_config` file

```python
config_string = rospy.get_param("/traffic_light_config")
self.config = yaml.load(config_string)
```


- your task for this portion of the project can be broken into two steps:
  - use the vehicle's location and the `(x,y)` coordinates for traffic lights to find the nearest visible traffic light ahead of the vehicle
    - this takes place in the `process_traffic_lights` method of `tl_detector.py`
    - you will want to use the `get_closest_waypoint` method to find the closest waypoints to the vehicle and lights
    - using these waypoint indices, you can determine which light is ahead of the vehicle along the list of waypoints
    - the code to publish the results of `process_traffic_lights` is written for you already in the `image_cb` method
  - use the camera image data to classify the color of the traffic light
    - the core functionality of this step takes place in the `get_light_state method` of `tl_detector.py`
    - there are a number of approaches you could take for this task
    - one of the simpler approaches is to train a deep learning classifier to classify the entire image as containing either a red light, yellow light, green light, or no light
    - one resource that's available to you is the traffic light's position in 3D space via the `vehicle/traffic_lights` topic

### Traffic Light Detection package files

- within the traffic light detection package, you will find the following:
  - `tl_detector.py`
    - this python file processes the incoming traffic light data and camera images
    - it uses the light classifier to get a color prediction, and publishes the location of any upcoming red lights
  - `tl_classifier.py`
    - this file contains the `TLClassifier` class
    - you can use this class to implement traffic light classification
    - for example, the `get_classification` method can take a camera image as input and return an ID corresponding to the color state of the traffic light in the image
    - note that it is not required for you to use this class--it only exists to help you break down the classification problem into more manageable chunks
    - also note that Carla currently has TensorFlow 1.3.0 installed--if you are using TensorFlow, please be sure to test your code with this version before submission
  - `traffic_light_config`
    - this config file contains information about the camera (such as focal length) and the 2D position of the traffic lights's stop line in world coordinates

### Helper Tool in the Simulator

- in order to help you acquire an accurate ground truth data source for the traffic light classifier, the Udacity simulator publishes the current color state of all traffic lights in the simulator to the `/vehicle/traffic_lights` topic in addition to the light location
- this state can be used to generate classified images or subbed into your solution to help you work on another single component of the node
- the state component of the topic won't be available when running your solution in real life so don't rely on it in the final submission
- however, you can still reference this topic in real life to get the 3D world position of the traffic light

### Using Different Classification Models for the Simulator and Site

- we will test your team's code both in the simulator and on the testing site
- due to differences in the appearance of the site and simulator traffic lights, using the same traffic light classification model for both might not be appropriate
- the `self.config` dictionary found in the `TLDetector` class of `tl_detector.py` contains an `is_site` boolean
- you can use this boolean to load a different classification model depending on the context

# Waypoint Updater Node (Full)

### Waypoint Updater Node Revisited

- once traffic light detection is working properly, you can incorporate the traffic light data into your waypoint updater node
- to do this, you will need to add a subscriber for the `/traffic_waypoint` topic and implement the `traffic_cb` callback for this subscriber


- there are several helpful methods that you can use:
  - `get_waypoint_velocity(self, waypoint)`: gets the linear velocity (x-direction) for a single waypoint
  - `set_waypoint_velocity(self, waypoints, waypoint, velocity)`: sets the linear velocity (x-direction) for a single waypoint in a list of waypoints
    - here, waypoints is a list of waypoints, waypoint is a waypoint index in the list, and velocity is the desired velocity
  - `distance(self, waypoints, wp1, wp2)`: computes the distance between two waypoints in a list along the piecewise linear arc connecting all waypoints between the two
    - here, `waypoints` is a list of waypoints, and `wp1` and `wp2` are the indices of two waypoints in the list
    - this method may be helpful in determining the velocities for a sequence of waypoints leading up to a red light (the velocities should gradually decrease to zero starting some distance from the light)


- to accomplish this part of the project successfully, you will need to adjust the target velocities for the waypoints leading up to red traffic lights or other obstacles in order to bring the vehicle to a smooth and full stop
- you should aim to have a smooth decrease in velocity leading up to the stopping point


- it will be up to you to determine what the deceleration should be for the vehicle, and how this deceleration corresponds to waypoint target velocities
- as in the Path Planning project, acceleration should not exceed 10 m/s^2 and jerk should not exceed 10 m/s^3


### Important:

- be sure to limit the top speed of the vehicle to the km/h velocity set by the `velocity` rosparam in `waypoint_loader`
- reviewers will test on the simulator with an adjusted top speed prior to testing on Carla

# Getting Feedback

### Getting and using rosbag feedback

- once your team submits the project, the Team Lead’s submission will first be tested using a simulator to ensure that the vehicle is following waypoints and adhering to the km/h velocity limit set by the velocity rosparam in waypoint_loader
- once this is verified, our capstone reviewer will take your project to the Udacity test track for real-world testing on Carla!


- after both the simulator test and the Carla test are complete, your Team Lead will receive a link to download the team’s feedback in the form of log files and a ROS bag recording of Carla on our test track
- note that it typically takes up to a week to receive feedback for this project, so plan accordingly


- the ROS bag file contains all messages passed on all topics in ROS while Carla is driving, providing your team with a full recording of all available data (including camera data) created at the time of testing
- note that the download link to this feedback expires after 20 days, so Team Leads should be sure to download all data and feedback promptly


- to replay the ROS bag using the workspace, use the following steps:
  - download the rviz config file [here](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/default.rviz)
  - open a terminal and start `roscore`
  - open another terminal and run `rosbag play -l /path/to/your.bag`
  - open one more terminal and run `rviz`
    - you can change the RViz configuration to the Udacity download by navigating to your config file from `File > Open Config in RViz`
    - alternatively, if you'd like to make the Udacity config file your default, you can replace the rviz config file found in `~/.rviz/default.rviz`


- while the bag is playing, you can use ROS commands to list or echo topics, grep and pipe messages, or use any other debugging techniques that might be useful
- this may be helpful, for example, if you wanted to see the ROS messages sent over the `/vehicle/throttle_cmd` topic as the vehicle was driving around the lot
- if you've used logging in your ROS nodes, remember that all log messages except for debugging messages are published to the `/rosout` topic by default, so you will be able to view logging messages using the ROS bag as well
- if you'd like debugging messages to also show up in the logs, be sure to set the `log_level` when initializing your ROS nodes:
```shell
rospy.init_node('my_node', log_level=rospy.DEBUG)
```